In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import kerastuner as kt
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn import metrics

In [ ]:
TrainDx = pd.read_csv('../input/prerocessed-danger-nbaiot/TrainDx.csv')
TrainDy = pd.read_csv('../input/prerocessed-danger-nbaiot/TrainDy.csv')
ValDx = pd.read_csv('../input/prerocessed-danger-nbaiot/ValDx.csv')
ValDy = pd.read_csv('../input/prerocessed-danger-nbaiot/ValDy.csv')

In [ ]:
def build_model(hp):
    model = Sequential()
    counter=0
    for i in range(hp.Int('num_layers', min_value=1, max_value=5)):
        if counter == 0:
            model.add(LSTM(hp.Int('units'+str(i), min_value=8, max_value=48, step=8), activation=hp.Choice('activation'+str(i), values=['relu', 'tanh', 'sigmoid']), input_shape=(TrainDx.shape[1], 1)))
        else:
            model.add(Dense(hp.Int('units'+str(i), min_value=8, max_value=48, step=8), activation=hp.Choice('activation'+str(i), values=['relu', 'tanh', 'sigmoid'])))
        counter += 1
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
tuner= kt.RandomSearch(build_model, objective='val_accuracy', max_trials=3)

In [ ]:
TrainDxRNN = TrainDx.values.reshape((TrainDx.shape[0], TrainDx.shape[1], 1))
tuner.search(TrainDxRNN, TrainDy, epochs=5, validation_data = (ValDx, ValDy))

In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]
best_model

In [ ]:
best_model.summary()

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Number of layers are: " + str(best_hps.get('num_layers')))
print("Layer1 is "+best_hps.get('activation1')+" of size "+str(best_hps.get('units0')))

## Accuracies

In [ ]:
def accFinder(X, y, treshold):
    return metrics.accuracy_score(pd.DataFrame((best_model.predict(X)>treshold).astype(int)), y)

In [ ]:
acc_card = []

In [ ]:
acc_card.append(accFinder(TrainDx, TrainDy, 0.5))

In [ ]:
acc_card.append(accFinder(ValDx, ValDy, 0.5))

In [ ]:
TestDx = pd.read_csv('../input/prerocessed-danger-nbaiot/TestDx.csv')
TestDy = pd.read_csv('../input/prerocessed-danger-nbaiot/TestDy.csv')

In [ ]:
acc_card.append(accFinder(TestDx, TestDy, 0.5))

In [ ]:
# Create a list of indices for the x-axis
indices = range(len(acc_card))
# Create the bar graph
bars = plt.bar(indices, acc_card)
# Optionally, you can set the labels for the x-axis to be something other than the indices
labels = ['Train', 'Val', 'Test']
plt.xticks(indices, labels)
# Labeling the axes
plt.xlabel('Datasets')
plt.ylabel('Accuracies')
# Giving a title to the graph
plt.title('Accuracies for RNN')
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x(), yval, '{:.10f}'.format(yval), va='bottom') # va: vertical alignment
plt.show()

## Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
TestPreds = (best_model.predict(TestDx)>0.5).astype(int)

In [ ]:
# 0.5
cm = pd.DataFrame(confusion_matrix(pd.DataFrame(TestPreds), TestDy, labels=[0,1]))
cm

In [ ]:
sns.heatmap(cm, annot=True, cmap='Blues')
plt.xlabel('Actual')
plt.ylabel('Predicted');

In [ ]:
def recall(type, TestPreds, TestDy, labels):
    confMat = confusion_matrix(TestPreds, TestDy, labels=labels)
    true = confMat[type][type]
    total = 0
    for i in range(len(labels)):
        total += confMat[i][type]
    return true/total

def precision(type, TestPreds, TestDy, labels):
    confMat = confusion_matrix(TestPreds, TestDy, labels=labels)
    true = confMat[type][type]
    total = 0
    for i in range(len(labels)):
        total += confMat[type][i]
    return true/total

def specificity(type, TestPreds, TestDy, labels):
    confMat = confusion_matrix(TestPreds, TestDy, labels=labels)
    true = 0
    for i in range(len(labels)):
        if(i==type):
            continue;
        true += confMat[i][i]
    total = 0
    for j in range(len(labels)):
        if(j==type):
            continue
        for i in range(len(labels)):
            total += confMat[i][j]
    return true/total

In [ ]:
labels=[0,1]

In [ ]:
for i in range(len(labels)):
    print('recall of class '+str(i)+': '+str(recall(i, TestPreds, TestDy, [0,1])))

In [ ]:
for i in range(len(labels)):
    print('precision of class '+str(i)+': '+str(precision(i, TestPreds, TestDy, [0,1])))

In [ ]:
for i in range(len(labels)):
    print('specificity of class '+str(i)+': '+str(specificity(i, TestPreds, TestDy, [0,1])))

## Saving the model

In [ ]:
import pickle
with open('RNNDanger.pkl', 'wb') as file:
    pickle.dump(best_model, file)

In [ ]:
import joblib
dt = {
    'RNNDanger':best_model,
}
joblib.dump(dt, 'RNNDanger.joblib')